# Chatbot with External Memory - Practice Exercises

## Overview
This notebook provides hands-on exercises to practice building conversational AI systems with persistent external memory using database checkpointers. You'll learn to create chatbots that maintain conversation state across sessions and restarts.

## Learning Objectives
By the end of these exercises, you will:
- Understand external database checkpointers vs in-memory storage
- Set up and use SQLite checkpointers for persistent conversation memory
- Build chatbots that survive application restarts and maintain long-term memory
- Implement conversation threading and user session management
- Create backup and recovery systems for conversation data
- Design scalable memory architectures for production chatbots

## Prerequisites
- Completed the chatbot-external-memory.ipynb tutorial
- Understanding of conversation summarization concepts
- Basic knowledge of databases and data persistence

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph-checkpoint-sqlite langchain_core langgraph langchain_openai

In [ ]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

# Optional: Set up LangSmith for tracing
# _set_env("LANGSMITH_API_KEY")
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_PROJECT"] = "langchain-academy"

## Exercise 1: Basic SQLite Persistent Chatbot

### Task
Create a chatbot that uses SQLite for persistent memory, allowing conversations to continue across application restarts.

### TODO: Set up SQLite checkpointer and basic persistent chatbot

In [ ]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage, AIMessage
from langgraph.graph import MessagesState, StateGraph, START, END
from IPython.display import Image, display
from typing import Literal

# TODO: Set up SQLite database connection
# Create a local database file for persistent storage
db_path = "persistent_chatbot.db"
# TODO: Create SQLite connection
# TODO: Create SqliteSaver checkpointer

# Initialize model
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# TODO: Define persistent chatbot state (reuse SummarizationState from previous exercise)
class PersistentChatState(MessagesState):
    # TODO: Add summary and user_id fields
    pass

In [ ]:
# TODO: Implement persistent chat nodes (similar to previous summarization exercise)
def persistent_chat_node(state: PersistentChatState):
    print(f"Processing chat for user: {state.get('user_id', 'unknown')}")
    
    # TODO: Get summary and messages
    summary = state.get("summary", "")
    messages = state["messages"]
    
    # TODO: Add summary to system message if it exists
    # TODO: Get response from model
    # TODO: Return response
    pass

# TODO: Implement persistent summarization
def persistent_summarize(state: PersistentChatState):
    print("Creating persistent summary...")
    
    messages = state["messages"]
    current_summary = state.get("summary", "")
    
    # TODO: Create or extend summary
    # TODO: Remove old messages except last 2
    # TODO: Return updated summary and cleaned messages
    pass

# TODO: Implement decision function
def should_summarize_persistent(state: PersistentChatState) -> Literal["summarize", "continue"]:
    # TODO: Return "summarize" if more than 6 messages, otherwise "continue"
    pass

In [ ]:
# TODO: Build persistent chatbot graph
builder_persistent = StateGraph(PersistentChatState)
# TODO: Add nodes and edges (similar structure to previous summarization exercise)
# TODO: Compile with SQLite checkpointer

display(Image(graph_persistent.get_graph().draw_mermaid_png()))

In [ ]:
# TODO: Test persistent chatbot - Part 1
user_id = "alice_123"
config = {"configurable": {"thread_id": f"persistent_chat_{user_id}"}}

# Start a conversation
conversation_part_1 = [
    "Hi, I'm Alice and I'm working on a Python web application.",
    "I'm using FastAPI and need help with database integration.",
    "Should I use SQLAlchemy or a simpler solution?",
    "I'm storing user profiles and transaction data."
]

print("=== Conversation Part 1 ===")
for i, user_input in enumerate(conversation_part_1, 1):
    print(f"\nTurn {i}:")
    print(f"Alice: {user_input}")
    
    result = graph_persistent.invoke(
        {"messages": [HumanMessage(content=user_input)], "user_id": user_id},
        config
    )
    
    last_message = result["messages"][-1]
    print(f"Assistant: {last_message.content[:100]}...")
    
    if result.get("summary"):
        print(f"Summary: {result['summary'][:100]}...")

print(f"\n=== Conversation state persisted to database ===")
print(f"Messages in conversation: {len(result['messages'])}")

In [ ]:
# TODO: Test persistence by "restarting" the application
print("\n=== Simulating Application Restart ===")
print("Creating new graph instance with same database...")

# TODO: Create new connection and checkpointer (simulating app restart)
new_conn = sqlite3.connect(db_path, check_same_thread=False)
new_memory = SqliteSaver(new_conn)

# TODO: Create new graph instance
graph_persistent_restarted = builder_persistent.compile(checkpointer=new_memory)

# TODO: Continue conversation with same thread_id
conversation_part_2 = [
    "I'm back! Do you remember what we were discussing?",
    "Great! Can you help me set up the database models?",
    "What about handling database migrations?"
]

print("\n=== Conversation Part 2 (After Restart) ===")
for i, user_input in enumerate(conversation_part_2, 1):
    print(f"\nTurn {i}:")
    print(f"Alice: {user_input}")
    
    result = graph_persistent_restarted.invoke(
        {"messages": [HumanMessage(content=user_input)], "user_id": user_id},
        config
    )
    
    last_message = result["messages"][-1]
    print(f"Assistant: {last_message.content[:100]}...")
    
    if result.get("summary"):
        print(f"Summary: {result['summary'][:100]}...")

print("\n=== Persistence Test Complete! ===")

## Exercise 2: Multi-User Persistent Chat System

### Task
Create a system that can handle multiple users with isolated conversation threads, each with their own persistent memory.

### TODO: Implement multi-user persistent system

In [ ]:
from typing import Dict, List, Optional
import uuid
from datetime import datetime

# TODO: Define multi-user state with user management
class MultiUserChatState(MessagesState):
    summary: str
    user_id: str
    # TODO: Add user_profile, conversation_metadata, session_info
    pass

# TODO: Implement user profile management
class UserProfileManager:
    def __init__(self, db_connection):
        self.conn = db_connection
        self.create_user_tables()
    
    def create_user_tables(self):
        # TODO: Create tables for user profiles and conversation metadata
        cursor = self.conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS user_profiles (
                user_id TEXT PRIMARY KEY,
                username TEXT,
                preferences TEXT,
                created_at TEXT,
                last_active TEXT
            )
        """)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS conversation_metadata (
                thread_id TEXT PRIMARY KEY,
                user_id TEXT,
                conversation_title TEXT,
                created_at TEXT,
                last_message_at TEXT,
                message_count INTEGER
            )
        """)
        self.conn.commit()
    
    def create_or_update_user(self, user_id: str, username: str, preferences: Dict = None):
        # TODO: Create or update user profile
        pass
    
    def get_user_conversations(self, user_id: str) -> List[Dict]:
        # TODO: Get all conversations for a user
        pass

# TODO: Set up multi-user database
multiuser_db_path = "multiuser_chatbot.db"
multiuser_conn = sqlite3.connect(multiuser_db_path, check_same_thread=False)
multiuser_memory = SqliteSaver(multiuser_conn)
profile_manager = UserProfileManager(multiuser_conn)

In [ ]:
# TODO: Implement multi-user chat nodes
def multiuser_chat_node(state: MultiUserChatState):
    print(f"Processing chat for user: {state['user_id']}")
    
    # TODO: Get user profile and customize response accordingly
    # TODO: Add user context to system message
    # TODO: Generate personalized response
    pass

def update_user_activity(state: MultiUserChatState):
    print(f"Updating activity for user: {state['user_id']}")
    
    # TODO: Update user's last activity timestamp
    # TODO: Update conversation metadata
    # TODO: Track conversation statistics
    pass

def multiuser_summarize(state: MultiUserChatState):
    print(f"Creating personalized summary for user: {state['user_id']}")
    
    # TODO: Create user-specific summary
    # TODO: Consider user preferences in summarization style
    # TODO: Update conversation metadata
    pass

In [ ]:
# TODO: Build multi-user graph
builder_multiuser = StateGraph(MultiUserChatState)
# TODO: Add nodes with user activity tracking
# TODO: Include update_user_activity in the flow

graph_multiuser = builder_multiuser.compile(checkpointer=multiuser_memory)
display(Image(graph_multiuser.get_graph().draw_mermaid_png()))

In [ ]:
# TODO: Test multi-user system
# Create multiple users with different conversation threads
users = [
    {"user_id": "alice_dev", "username": "Alice", "interests": ["Python", "Web Development"]},
    {"user_id": "bob_data", "username": "Bob", "interests": ["Data Science", "Machine Learning"]},
    {"user_id": "charlie_mobile", "username": "Charlie", "interests": ["Mobile Apps", "React Native"]}
]

# TODO: Create user profiles
for user in users:
    profile_manager.create_or_update_user(
        user["user_id"], 
        user["username"], 
        {"interests": user["interests"]}
    )

# TODO: Simulate conversations for each user
conversations = {
    "alice_dev": [
        "Hi, I'm Alice. I'm building a REST API with Python.",
        "I need help with authentication and authorization.",
        "Should I use JWT tokens or session-based auth?"
    ],
    "bob_data": [
        "Hello, I'm Bob. I work with data science projects.",
        "I'm analyzing customer behavior data using pandas.",
        "What's the best way to handle missing values?"
    ],
    "charlie_mobile": [
        "Hey, I'm Charlie. I develop mobile applications.",
        "I'm using React Native for cross-platform development.",
        "How do I handle offline data synchronization?"
    ]
}

# TODO: Process conversations for each user
for user_id, messages in conversations.items():
    print(f"\n=== Conversation with {user_id} ===")
    config = {"configurable": {"thread_id": f"thread_{user_id}_{uuid.uuid4().hex[:8]}"}}
    
    for i, message in enumerate(messages, 1):
        print(f"\nTurn {i}:")
        print(f"User: {message}")
        
        result = graph_multiuser.invoke(
            {
                "messages": [HumanMessage(content=message)], 
                "user_id": user_id,
                "user_profile": users[[u["user_id"] for u in users].index(user_id)]
            },
            config
        )
        
        last_message = result["messages"][-1]
        print(f"Assistant: {last_message.content[:80]}...")

print("\n=== Multi-User Test Complete ===")

## Exercise 3: Conversation Backup and Recovery System

### Task
Implement a backup and recovery system for conversation data, including export/import functionality and data migration.

### TODO: Implement backup and recovery system

In [ ]:
import json
import shutil
from pathlib import Path
import zipfile
from datetime import datetime

class ConversationBackupManager:
    def __init__(self, db_path: str, backup_dir: str = "backups"):
        self.db_path = db_path
        self.backup_dir = Path(backup_dir)
        self.backup_dir.mkdir(exist_ok=True)
        
    def create_database_backup(self) -> str:
        """Create a full database backup."""
        # TODO: Create timestamped database backup
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_filename = f"chatbot_backup_{timestamp}.db"
        backup_path = self.backup_dir / backup_filename
        
        # TODO: Copy database file
        # TODO: Return backup path
        pass
    
    def export_conversations_json(self, user_id: str = None) -> str:
        """Export conversations to JSON format."""
        # TODO: Connect to database and extract conversation data
        # TODO: Convert to JSON format with metadata
        # TODO: Save to file and return path
        pass
    
    def import_conversations_json(self, json_file_path: str) -> bool:
        """Import conversations from JSON file."""
        # TODO: Load JSON data
        # TODO: Validate data format
        # TODO: Insert into database with conflict resolution
        pass
    
    def create_compressed_backup(self) -> str:
        """Create a compressed backup with metadata."""
        # TODO: Create ZIP archive with database and metadata
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_filename = f"chatbot_full_backup_{timestamp}.zip"
        backup_path = self.backup_dir / backup_filename
        
        # TODO: Add database file, user profiles, and metadata to ZIP
        pass
    
    def restore_from_backup(self, backup_path: str) -> bool:
        """Restore database from backup."""
        # TODO: Validate backup file
        # TODO: Restore database with confirmation
        # TODO: Handle different backup formats (DB, ZIP, JSON)
        pass
    
    def list_backups(self) -> List[Dict]:
        """List available backups with metadata."""
        # TODO: Scan backup directory
        # TODO: Extract metadata from backup files
        # TODO: Return sorted list with creation dates and sizes
        pass

# TODO: Initialize backup manager
backup_manager = ConversationBackupManager(multiuser_db_path)

In [ ]:
# TODO: Test backup functionality
print("=== Testing Backup System ===")

# Create database backup
print("\n1. Creating database backup...")
db_backup_path = backup_manager.create_database_backup()
print(f"Database backup created: {db_backup_path}")

# Export conversations to JSON
print("\n2. Exporting conversations to JSON...")
json_backup_path = backup_manager.export_conversations_json(user_id="alice_dev")
print(f"JSON export created: {json_backup_path}")

# Create compressed backup
print("\n3. Creating compressed backup...")
zip_backup_path = backup_manager.create_compressed_backup()
print(f"Compressed backup created: {zip_backup_path}")

# List all backups
print("\n4. Listing all backups...")
backups = backup_manager.list_backups()
for backup in backups:
    print(f"  - {backup['filename']}: {backup['size']} bytes, created {backup['created']}")

print("\n=== Backup System Test Complete ===")

In [ ]:
# TODO: Test recovery functionality
print("\n=== Testing Recovery System ===")

# TODO: Simulate data loss by creating a new database
recovery_test_db = "recovery_test.db"
recovery_conn = sqlite3.connect(recovery_test_db, check_same_thread=False)
recovery_memory = SqliteSaver(recovery_conn)

print("\n1. Created empty database for recovery test")

# TODO: Test restoration from backup
print("\n2. Restoring from backup...")
restoration_success = backup_manager.restore_from_backup(db_backup_path)
print(f"Restoration {'successful' if restoration_success else 'failed'}")

# TODO: Verify data integrity after restoration
print("\n3. Verifying data integrity...")
# Create new graph with restored database
recovery_graph = builder_multiuser.compile(checkpointer=recovery_memory)

# Test with existing conversation thread
test_config = {"configurable": {"thread_id": "thread_alice_dev_test"}}
test_state = recovery_graph.get_state(test_config)
print(f"Recovered conversation has {len(test_state.values.get('messages', []))} messages")

print("\n=== Recovery System Test Complete ===")

# Cleanup
recovery_conn.close()
if Path(recovery_test_db).exists():
    Path(recovery_test_db).unlink()

## Exercise 4: Scalable Memory Architecture

### Task
Design a scalable memory architecture that can handle high-throughput conversations with automatic cleanup and optimization.

### TODO: Implement scalable memory system

In [ ]:
import threading
import time
from concurrent.futures import ThreadPoolExecutor
from typing import Callable

class ScalableMemoryManager:
    def __init__(self, db_path: str, max_connections: int = 10):
        self.db_path = db_path
        self.max_connections = max_connections
        self.connection_pool = []
        self.cleanup_thread = None
        self.is_running = True
        
        # TODO: Initialize connection pool
        self.init_connection_pool()
        
        # TODO: Start background cleanup thread
        self.start_cleanup_thread()
    
    def init_connection_pool(self):
        """Initialize connection pool for concurrent access."""
        # TODO: Create pool of database connections
        for i in range(self.max_connections):
            conn = sqlite3.connect(self.db_path, check_same_thread=False)
            conn.execute("PRAGMA journal_mode=WAL")  # Enable WAL mode for better concurrency
            self.connection_pool.append(conn)
    
    def get_connection(self) -> sqlite3.Connection:
        """Get connection from pool."""
        # TODO: Implement connection pool management
        # In a real implementation, use proper connection pooling
        return self.connection_pool[threading.current_thread().ident % len(self.connection_pool)]
    
    def start_cleanup_thread(self):
        """Start background thread for automatic cleanup."""
        # TODO: Implement background cleanup thread
        def cleanup_worker():
            while self.is_running:
                try:
                    self.cleanup_old_conversations()
                    self.optimize_database()
                    time.sleep(3600)  # Run every hour
                except Exception as e:
                    print(f"Cleanup error: {e}")
        
        self.cleanup_thread = threading.Thread(target=cleanup_worker, daemon=True)
        self.cleanup_thread.start()
    
    def cleanup_old_conversations(self, days_old: int = 30):
        """Remove conversations older than specified days."""
        # TODO: Implement cleanup logic
        conn = self.get_connection()
        cursor = conn.cursor()
        
        # TODO: Delete old conversation data
        # TODO: Preserve important conversations marked by users
        pass
    
    def optimize_database(self):
        """Optimize database performance."""
        # TODO: Run database optimization commands
        conn = self.get_connection()
        conn.execute("VACUUM")
        conn.execute("ANALYZE")
        conn.commit()
    
    def get_memory_stats(self) -> Dict:
        """Get memory usage statistics."""
        # TODO: Calculate database size, conversation counts, etc.
        conn = self.get_connection()
        cursor = conn.cursor()
        
        # Get database size
        db_size = Path(self.db_path).stat().st_size
        
        # TODO: Get conversation statistics
        # TODO: Get active users count
        # TODO: Return comprehensive stats
        pass
    
    def shutdown(self):
        """Shutdown memory manager gracefully."""
        self.is_running = False
        if self.cleanup_thread:
            self.cleanup_thread.join(timeout=5)
        
        # Close all connections
        for conn in self.connection_pool:
            conn.close()

# TODO: Initialize scalable memory manager
scalable_db_path = "scalable_chatbot.db"
memory_manager = ScalableMemoryManager(scalable_db_path)

# Create checkpointer with connection from pool
scalable_memory = SqliteSaver(memory_manager.get_connection())

In [ ]:
# TODO: Test scalable system with concurrent conversations
def simulate_user_conversation(user_id: str, message_count: int = 5):
    """Simulate a user conversation for load testing."""
    config = {"configurable": {"thread_id": f"load_test_{user_id}_{uuid.uuid4().hex[:8]}"}}
    
    messages = [
        f"Hi, I'm user {user_id}. I have a question about programming.",
        "Can you help me with Python best practices?",
        "What about error handling strategies?",
        "How do I write maintainable code?",
        "Thanks for the help!"
    ]
    
    # Use connection from pool
    local_memory = SqliteSaver(memory_manager.get_connection())
    local_graph = builder_multiuser.compile(checkpointer=local_memory)
    
    for i, message in enumerate(messages[:message_count]):
        try:
            result = local_graph.invoke(
                {
                    "messages": [HumanMessage(content=message)],
                    "user_id": user_id
                },
                config
            )
            time.sleep(0.1)  # Small delay between messages
        except Exception as e:
            print(f"Error in conversation {user_id}: {e}")

# TODO: Run concurrent conversation simulation
print("=== Testing Scalable Memory System ===")
print("\n1. Running concurrent conversation simulation...")

# Create multiple concurrent conversations
num_users = 20
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [
        executor.submit(simulate_user_conversation, f"user_{i}", 3)
        for i in range(num_users)
    ]
    
    # Wait for all conversations to complete
    for future in futures:
        future.result()

print(f"\n2. Completed {num_users} concurrent conversations")

# TODO: Check memory statistics
print("\n3. Memory statistics:")
stats = memory_manager.get_memory_stats()
for key, value in stats.items():
    print(f"   {key}: {value}")

print("\n=== Scalable Memory Test Complete ===")

## Exercise 5: Production Deployment Considerations

### Task
Implement production-ready features including monitoring, logging, and error handling for persistent memory systems.

### TODO: Implement production features

In [ ]:
import logging
from typing import Any, Dict
import traceback
from functools import wraps

class ProductionMemorySystem:
    def __init__(self, db_path: str):
        self.db_path = db_path
        self.setup_logging()
        self.setup_monitoring()
        self.connection = sqlite3.connect(db_path, check_same_thread=False)
        self.checkpointer = SqliteSaver(self.connection)
        
    def setup_logging(self):
        """Setup production logging."""
        # TODO: Configure structured logging
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler('chatbot_production.log'),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger('ProductionMemorySystem')
        
    def setup_monitoring(self):
        """Setup monitoring and metrics collection."""
        # TODO: Initialize metrics tracking
        self.metrics = {
            'conversations_created': 0,
            'messages_processed': 0,
            'errors': 0,
            'database_operations': 0
        }
        
    def log_operation(self, operation: str, user_id: str, success: bool, details: Dict = None):
        """Log operation with structured data."""
        # TODO: Implement structured logging
        log_data = {
            'operation': operation,
            'user_id': user_id,
            'success': success,
            'timestamp': datetime.now().isoformat()
        }
        if details:
            log_data.update(details)
        
        if success:
            self.logger.info(f"Operation successful: {log_data}")
        else:
            self.logger.error(f"Operation failed: {log_data}")
    
    def with_error_handling(self, operation_name: str):
        """Decorator for error handling and monitoring."""
        def decorator(func):
            @wraps(func)
            def wrapper(*args, **kwargs):
                try:
                    result = func(*args, **kwargs)
                    self.metrics['database_operations'] += 1
                    user_id = kwargs.get('user_id', 'unknown')
                    self.log_operation(operation_name, user_id, True)
                    return result
                except Exception as e:
                    self.metrics['errors'] += 1
                    user_id = kwargs.get('user_id', 'unknown')
                    self.log_operation(
                        operation_name, 
                        user_id, 
                        False, 
                        {'error': str(e), 'traceback': traceback.format_exc()}
                    )
                    # TODO: Implement fallback behavior
                    raise
            return wrapper
        return decorator
    
    @with_error_handling('conversation_processing')
    def process_conversation(self, messages: List, user_id: str, config: Dict) -> Dict:
        """Process conversation with full production monitoring."""
        # TODO: Implement conversation processing with monitoring
        self.metrics['messages_processed'] += len(messages)
        
        # Create production graph
        production_graph = builder_multiuser.compile(checkpointer=self.checkpointer)
        
        # Process with monitoring
        result = production_graph.invoke(
            {"messages": messages, "user_id": user_id},
            config
        )
        
        return result
    
    def health_check(self) -> Dict[str, Any]:
        """Comprehensive health check."""
        # TODO: Implement comprehensive health monitoring
        health_status = {
            'status': 'healthy',
            'database_connection': 'ok',
            'metrics': self.metrics.copy(),
            'database_size_mb': Path(self.db_path).stat().st_size / (1024 * 1024)
        }
        
        try:
            # Test database connection
            self.connection.execute("SELECT 1")
        except Exception as e:
            health_status['status'] = 'unhealthy'
            health_status['database_connection'] = f'error: {str(e)}'
        
        return health_status
    
    def get_performance_metrics(self) -> Dict[str, Any]:
        """Get performance metrics."""
        # TODO: Calculate performance metrics
        return {
            'total_operations': self.metrics['database_operations'],
            'error_rate': self.metrics['errors'] / max(1, self.metrics['database_operations']),
            'messages_per_hour': self.metrics['messages_processed'],  # Simplified
            'database_size_mb': Path(self.db_path).stat().st_size / (1024 * 1024)
        }

# TODO: Initialize production system
production_db_path = "production_chatbot.db"
production_system = ProductionMemorySystem(production_db_path)

In [ ]:
# TODO: Test production system
print("=== Testing Production Memory System ===")

# Test health check
print("\n1. Health Check:")
health = production_system.health_check()
for key, value in health.items():
    print(f"   {key}: {value}")

# Test error handling
print("\n2. Testing error handling...")
try:
    result = production_system.process_conversation(
        [HumanMessage(content="Test production system")],
        user_id="production_test_user",
        config={"configurable": {"thread_id": "production_test_thread"}}
    )
    print("   Conversation processed successfully")
except Exception as e:
    print(f"   Error handled: {e}")

# Check performance metrics
print("\n3. Performance Metrics:")
metrics = production_system.get_performance_metrics()
for key, value in metrics.items():
    print(f"   {key}: {value}")

# Test logging by checking log file
print("\n4. Recent log entries:")
try:
    with open('chatbot_production.log', 'r') as f:
        lines = f.readlines()
        for line in lines[-3:]:
            print(f"   {line.strip()}")
except FileNotFoundError:
    print("   Log file not found")

print("\n=== Production System Test Complete ===")

## Challenge Exercise: Distributed Memory System

### Task
Design a distributed memory system that can scale across multiple database instances and handle conversation sharding.

### TODO: Implement distributed memory architecture

In [ ]:
import hashlib
from typing import List, Tuple

class DistributedMemorySystem:
    def __init__(self, shard_configs: List[Dict]):
        """Initialize distributed memory with multiple database shards."""
        self.shards = []
        self.num_shards = len(shard_configs)
        
        # TODO: Initialize multiple database shards
        for i, config in enumerate(shard_configs):
            shard_path = config.get('db_path', f'shard_{i}.db')
            conn = sqlite3.connect(shard_path, check_same_thread=False)
            checkpointer = SqliteSaver(conn)
            
            self.shards.append({
                'id': i,
                'connection': conn,
                'checkpointer': checkpointer,
                'config': config
            })
    
    def get_shard_for_user(self, user_id: str) -> int:
        """Determine which shard to use for a given user."""
        # TODO: Implement consistent hashing for user-to-shard mapping
        user_hash = int(hashlib.md5(user_id.encode()).hexdigest(), 16)
        return user_hash % self.num_shards
    
    def get_checkpointer_for_user(self, user_id: str):
        """Get the appropriate checkpointer for a user."""
        shard_id = self.get_shard_for_user(user_id)
        return self.shards[shard_id]['checkpointer']
    
    def migrate_user_conversations(self, user_id: str, from_shard: int, to_shard: int):
        """Migrate user conversations between shards."""
        # TODO: Implement conversation migration logic
        pass
    
    def get_distributed_stats(self) -> Dict:
        """Get statistics across all shards."""
        # TODO: Aggregate statistics from all shards
        total_stats = {
            'total_shards': self.num_shards,
            'shard_stats': []
        }
        
        for shard in self.shards:
            # TODO: Get stats from each shard
            shard_stat = {
                'shard_id': shard['id'],
                'size_mb': 0,  # TODO: Calculate actual size
                'conversations': 0  # TODO: Count conversations
            }
            total_stats['shard_stats'].append(shard_stat)
        
        return total_stats

# TODO: Initialize distributed system
shard_configs = [
    {'db_path': 'distributed_shard_0.db', 'region': 'us-east'},
    {'db_path': 'distributed_shard_1.db', 'region': 'us-west'},
    {'db_path': 'distributed_shard_2.db', 'region': 'eu-west'}
]

distributed_system = DistributedMemorySystem(shard_configs)

print("Distributed memory system initialized with sharding")
print("This demonstrates the architecture - implement the TODOs for full functionality!")

## Cleanup and Summary

In [ ]:
# TODO: Cleanup resources
print("=== Cleaning up resources ===")

# Close database connections
try:
    multiuser_conn.close()
    memory_manager.shutdown()
    production_system.connection.close()
    
    for shard in distributed_system.shards:
        shard['connection'].close()
        
    print("All database connections closed successfully")
except Exception as e:
    print(f"Error during cleanup: {e}")

print("\n=== Exercise Complete ===")

## Summary

In these exercises, you've practiced:
- Setting up SQLite checkpointers for persistent conversation memory
- Building multi-user chat systems with isolated conversation threads
- Implementing backup and recovery systems for conversation data
- Creating scalable memory architectures with connection pooling and optimization
- Adding production-ready features like monitoring, logging, and error handling
- Designing distributed memory systems with sharding

Key takeaways:
- **External Memory**: Database checkpointers provide persistent conversation state across restarts
- **SQLite Benefits**: Fast, lightweight, and perfect for single-application deployments
- **Multi-User Systems**: Proper thread isolation and user management are essential
- **Backup Strategies**: Multiple backup formats (DB, JSON, compressed) serve different recovery needs
- **Scalability**: Connection pooling, background cleanup, and optimization are crucial for production
- **Production Features**: Monitoring, logging, and error handling are non-negotiable for real systems
- **Distributed Systems**: Sharding enables horizontal scaling but adds complexity

These external memory techniques are essential for building production-ready conversational AI systems that can:
- Maintain long-term user relationships
- Handle high concurrency
- Recover from failures gracefully
- Scale horizontally when needed
- Provide reliable service in production environments

You've now completed all Module 2 exercises covering state management, schemas, reducers, memory, and message handling in LangGraph. These foundational concepts will serve you well as you build more advanced conversational AI systems!